In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Подготовим данные. Избавимся от лишних столбцов и строк

In [3]:
import pandas as pd
import json

In [4]:
sessionsDF = pd.read_csv('/content/drive/My Drive/Colab Notebooks/SimilarMusicians/data/relations/sessions.idomaar', sep="\t", header=None)

sessionsDF.drop([0,1,2], axis = 1, inplace = True)
sessionsDF.drop(sessionsDF.tail(2000000).index, inplace = True)


sessionsDF[3] = pd.Series(map(lambda s: list(map(lambda k : k['id'] ,json.loads(('[' + s + ']').replace('} ', '},'))[1]['objects'])), sessionsDF[3]))
sessionsDF.drop(sessionsDF[sessionsDF[3].map(len) < 6].index, inplace = True)
sessionsDF.drop(sessionsDF[sessionsDF[3].map(len) > 10].index, inplace = True)


sessionsDF


,3
7,"[150645, 150616, 657838, 3208711, 466151, 1539..."
9,"[3779365, 338599, 3214594, 3477941, 3678228, 5..."
15,"[549667, 549673, 3848461, 1632002, 1632005, 44..."
17,"[2942161, 2942215, 2942143, 2942029, 2942051, ..."
29,"[2554273, 133135, 1519407, 2311961, 2343670, 2..."
...,...
764431,"[2143404, 3358746, 3358746, 1559247, 1559160, ..."
764434,"[59930, 60079, 60224, 60171, 60058, 60002, 601..."
764436,"[800770, 800769, 800760, 800773, 800751, 80077..."
764448,"[3164059, 3164067, 3164056, 3164069, 3164048, ..."


In [5]:
personsDF = pd.read_csv('/content/drive/My Drive/Colab Notebooks/SimilarMusicians/data/entities/persons.idomaar', sep="\t", header=None)

personsDF['name'] = pd.Series(map(lambda s: json.loads(s)['name'], personsDF[3]))

personsDF.drop([0,2,3,4], axis=1, inplace=True)
personsDF.rename(columns={1:'ID'},inplace=True)
personsDF.set_index('ID',inplace=True)

personsDF.sort_values(by=['ID'], inplace=True)
personsDF = personsDF[~personsDF.index.duplicated()]
personsDF

,name
ID,
0,000003+Music+Instructor
1,00-01
2,0005.+Overkill
3,000C+Tony+Dize
4,000+Oscarcito
...,...
595136,Kwajbasket+&+Gimu
595137,David+Granha+&+John+Axion
595138,"Tam+Cooper,+Will+Saul"


In [7]:
tracksDF = pd.read_csv('/content/drive/My Drive/Colab Notebooks/SimilarMusicians/data/entities/tracks.idomaar', sep="\t", header=None)

tracksDF.drop_duplicates(keep='first',inplace=True) 
tracksDF = tracksDF.reset_index(drop=True)


tracksDF['name'] = pd.Series(map(lambda s: json.loads(s)['name'], tracksDF[3]))
tracksDF['playcount'] = pd.Series(map(lambda s: json.loads(s)['playcount'], tracksDF[3]))

tracksDF['artists'] = pd.Series(map(lambda s: json.loads(s)['artists'], tracksDF[4]))
tracksDF['albums'] = pd.Series(map(lambda s: json.loads(s)['albums'], tracksDF[4]))
tracksDF['tags'] = pd.Series(map(lambda s: json.loads(s)['tags'], tracksDF[4]))
tracksDF['artist_id'] = pd.Series(map(lambda s: s[0]['id'], tracksDF['artists']))

tracksDF.drop([0,2,3,4], axis=1, inplace=True)
tracksDF.rename(columns={1:'ID'},inplace=True)
tracksDF.set_index('ID', inplace = True)

tracksDF.sort_values(by=['ID'], inplace=True)
# tracksDF = tracksDF.drop(tracksDF[tracksDF.playcount.isnull() == True].index)
tracksDF.drop(['playcount','albums','tags','artists','name'], axis=1, inplace=True)
# tracksDF.drop_duplicates(keep='first',inplace=True) 
tracksDF = tracksDF[~tracksDF.index.duplicated()]


tracksDF


,artist_id
ID,
0,0
1,1
2,2
3,3
4,4
...,...
5023104,595139
5023105,187223
5023106,595140


In [8]:
def tracks_to_artists(session):
  artists = []
  for i in session:
    artists.append(tracksDF.loc[i].artist_id)

  return list(dict.fromkeys(artists))


sessionsDF['artists'] = pd.Series(map(lambda s: tracks_to_artists(s) , sessionsDF[3]))
sessionsDF = sessionsDF[(sessionsDF.artists.isnull() == False)][sessionsDF[(sessionsDF.artists.isnull() == False)].artists.map(len) > 2]

Из полученных датафреймов составим датасет

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset


In [20]:
import random

class ArtistsNegativeSampling(Dataset):

  def __init__(self, data, num_negative_samples, num_contexts=30000):

    self.data = data
    self.num_negative_samples = int(num_negative_samples)
    self.num_contexts = num_contexts
    self.num_tokens = personsDF.count()
    self.samples = None

  
  def generate_dataset(self):
    
    self.samples = []
    for i in range(self.num_contexts):
      for j, artist in enumerate(self.data.iloc[i]):
        for k in range(-2,3):
          if ((j+k >= 0)&(j+k<len(self.data.iloc[i]))&(k!=0)):
            sample = [artist, self.data.iloc[i][j+k]]
            self.samples.append(sample)


  def __len__(self):
    return len(self.samples)

  
  def __getitem__(self, index):
    
    target_index = self.samples[index][1]
    input_vector = self.samples[index][0]
    output_indices = torch.Tensor(random.sample(list(personsDF.index), self.num_negative_samples + 1))
    output_indices[0] = target_index
    output_target = torch.zeros(self.num_negative_samples + 1, dtype=torch.float32)
    output_target[0] = 1

    return (input_vector, output_indices, output_target)
        

dataset = ArtistsNegativeSampling(sessionsDF.artists,10,14000)
dataset.data

9                          [251758, 203788, 182910, 374201]
29        [53365, 54845, 369388, 18734, 240211, 305894, ...
35         [403228, 463053, 381263, 337804, 381710, 151838]
50         [174751, 153172, 152893, 148130, 123312, 420441]
64                           [84317, 313408, 86627, 191633]
                                ...                        
147636    [224085, 208629, 102735, 191173, 17608, 87334,...
147641                       [60557, 467679, 191173, 81215]
147650    [266242, 285932, 307718, 242068, 71903, 229135...
147652    [94077, 372203, 269536, 457713, 242522, 185676...
147656                              [254457, 79948, 210841]
Name: artists, Length: 14841, dtype: object

In [21]:
dataset.generate_dataset()

Задаем модель

In [22]:
num_tokens = personsDF.index[-1]


class ArtistsNegativeSamples(nn.Module):
    def __init__(self, num_tokens):
        super(ArtistsNegativeSamples, self).__init__()
        self.input = nn.Linear(num_tokens, 100, bias=False)
        self.output = nn.Linear(100, num_tokens, bias=False)
        
    def forward(self, input_index_batch, output_indices_batch):
        
        results = []

        for i, index in enumerate(input_index_batch):
          output_indices = output_indices_batch[i].long()
          pred = torch.mv(self.output.weight[output_indices,:],self.input.weight[:,index])
          results.append(pred)

        return torch.stack(results)

nn_model = ArtistsNegativeSamples(num_tokens)
nn_model.type(torch.FloatTensor)

ArtistsNegativeSamples(
  (input): Linear(in_features=595140, out_features=100, bias=False)
  (output): Linear(in_features=100, out_features=595140, bias=False)
)

Функция тренировки

In [26]:
def train_neg_sample(model, dataset, train_loader, optimizer, scheduler, num_epochs):    
    model = model.to(device)
    
    loss = nn.BCEWithLogitsLoss().type(torch.FloatTensor)
    loss_history = []
    train_history = []
    for epoch in range(num_epochs):
        if scheduler:
            scheduler.step()
            
        model.train() 
        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        
        
        dataset.generate_dataset()
        
        
        for i_step, (input_vector, output_indices, output_target) in enumerate(train_loader):
          print(i_step)
          prediction = model(input_vector.to(device), output_indices.to(device))
          loss_value = loss(prediction.to(device).float(), output_target.to(device).float())
          
          optimizer.zero_grad()
          loss_value.backward()
          optimizer.step()
          
          correct_samples += torch.sum((prediction>0) == output_target.byte().to(device))
          total_samples += 1
          
          loss_accum += loss_value

        ave_loss = loss_accum / (i_step + 1)
        train_accuracy = float(correct_samples) / total_samples   
        
        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        
        print("Average loss: %f, Train accuracy: %f" % (ave_loss, train_accuracy))
        
    return loss_history, train_history

In [27]:
device = torch.device("cuda")
nn_model.to(device)

ArtistsNegativeSamples(
  (input): Linear(in_features=595140, out_features=100, bias=False)
  (output): Linear(in_features=100, out_features=595140, bias=False)
)

Тренировка

In [ ]:
optimizer = optim.Adam(nn_model.parameters(), lr=30)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=1000)

loss_history, train_history = train_neg_sample(nn_model, dataset, train_loader, optimizer, scheduler, 1)

Достаем полученные ембедденги

In [32]:
nn_model.eval()

ArtistsNegativeSamples(
  (input): Linear(in_features=595140, out_features=100, bias=False)
  (output): Linear(in_features=100, out_features=595140, bias=False)
)

In [36]:
def extract_word_vectors(nn_model):
  return nn_model.input.weight.transpose(0, 1), nn_model.output.weight

In [37]:
trained_input_vectors, trained_output_vectors = extract_word_vectors(nn_model)

In [192]:
full_vectors = trained_input_vectors + trained_output_vectors

Находим похожих артистов

In [74]:
def sim_matrix(a, b, eps=1e-8):
    a_n, b_n = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]
    a_norm = a / torch.max(a_n, eps * torch.ones_like(a_n))
    b_norm = b / torch.max(b_n, eps * torch.ones_like(b_n))
    sim_mt = torch.mm(a_norm, b_norm.transpose(0, 1))
    return sim_mt

In [205]:
def findSimilarArtist(artist):

  similarNamesIndexes = personsDF[personsDF['name'].str.contains(artist)].index
  similarNamesVector = full_vectors[similarNamesIndexes].mean(0)

  simMatrix = sim_matrix(similarNamesVector.unsqueeze_(0),full_vectors)
  top20plus = torch.topk(simMatrix,len(similarNamesIndexes)+20).indices.squeeze(0)

  for i in similarNamesIndexes:
    top20plus = top20plus[top20plus != i]
  
  finalArtists = []
  top20plus = top20plus.cpu().tolist()
  
  for i in range(10):

    finalArtists.append(personsDF.loc[top20plus[i]]['name'])

  return finalArtists



In [213]:
findSimilarArtist('AC/DC')

['003',
 '002+%D0%98.%D0%9A%D1%83%D1%87%D0%B8%D0%BD',
 '002+Cari+Jodoh',
 '002C+Omega+El+Fuerte',
 '00-01',
 '000003+Music+Instructor',
 '0005.+Overkill',
 '000C+Tony+Dize',
 '001_Ariya',
 '001+Amber+Gomez']

In [216]:
findSimilarArtist('Drake')

['Space_Ci',
 'DJ+Feel-X',
 'Sunshine+State',
 'Nina+letar+Ufo',
 'Upside+Down+Miss+Jane',
 'Carlos+Kleiber',
 'Minerale',
 'Ynfl-X',
 'Emile+de+Becque',
 'Fallen+Rule']